In [256]:
import pickle
import tensorflow
import numpy as np
from numpy.linalg import norm
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input
from sklearn.neighbors import NearestNeighbors
import cv2
import pandas as pd 

from urllib.request import urlopen,urlretrieve
import imutils

import os

In [224]:
# High Resoultion images

feature_list = np.array(pickle.load(open('embeddings_highRES.pkl','rb')))
filenames = pickle.load(open('filenames_highRES.pkl','rb'))

In [225]:
# low Resoultion images

# feature_list = np.array(pickle.load(open('embeddings.pkl','rb')))
# filenames = pickle.load(open('filenames.pkl','rb'))

In [226]:
model = ResNet50(weights='imagenet',include_top=False,input_shape=(224,224,3))
model.trainable = False

In [227]:
model = tensorflow.keras.Sequential([
    model,
    GlobalMaxPooling2D()
])

In [334]:
img_url = "https://upload.wikimedia.org/wikipedia/commons/8/89/Capri_Pants_front_view.jpg"

In [335]:
img_link = urlretrieve(img_url)[0] # this image store in below location but this line of code read only .JPG images
img_link

'C:\\Users\\91798\\AppData\\Local\\Temp\\tmpb7h6u5t3'

In [336]:
img = image.load_img(img_link,target_size=(224,224))
img_array = image.img_to_array(img)
expanded_img_array = np.expand_dims(img_array, axis=0)
preprocessed_img = preprocess_input(expanded_img_array)
result = model.predict(preprocessed_img).flatten()
normalized_result = result / norm(result)

In [337]:
neighbors = NearestNeighbors(n_neighbors=5,algorithm='brute',metric='euclidean')
neighbors.fit(feature_list)

NearestNeighbors(algorithm='brute', metric='euclidean')

In [338]:
distances,indices = neighbors.kneighbors([normalized_result])

In [339]:
print(indices) # our predicted image IDs

[[29679 29647  1367  5692 40086]]


In [403]:
recomand_jpg = []
for i in indices[0]: 
    a = filenames[i]
    a = a.replace('/content/fashion-dataset/images/','')
    recomand_jpg.append(a)

In [404]:
recomand_jpg

['50669.jpg', '51394.jpg', '57085.jpg', '31231.jpg', '12671.jpg']

In [322]:
img_csv = pd.read_csv('images.csv')
img_csv

,filename,link
0,15970.jpg,http://assets.myntassets.com/v1/images/style/p...
1,39386.jpg,http://assets.myntassets.com/v1/images/style/p...
2,59263.jpg,http://assets.myntassets.com/v1/images/style/p...
3,21379.jpg,http://assets.myntassets.com/v1/images/style/p...
4,53759.jpg,http://assets.myntassets.com/v1/images/style/p...
...,...,...
44441,17036.jpg,http://assets.myntassets.com/v1/images/style/p...
44442,6461.jpg,http://assets.myntassets.com/v1/images/style/p...
44443,18842.jpg,http://assets.myntassets.com/v1/images/style/p...
44444,46694.jpg,http://assets.myntassets.com/v1/images/style/p...


In [405]:
final_img = []
for i in recomand_jpg:
    a = img_csv[img_csv['filename'] == i]['link'].values[0]
    final_img.append(a)

In [406]:
final_img

['http://assets.myntassets.com/v1/images/style/properties/3024d86ca01c50e8c3bf1890b9552eae_images.jpg',
 'http://assets.myntassets.com/v1/images/style/properties/c3cb7c416bb1d880b0e1b57f36b6fd53_images.jpg',
 'http://assets.myntassets.com/v1/images/style/properties/Elle-Women-Off-White---Orange-Floral-Print-Tulip-Skirt_79d11b62c2cbf86d94e6058a6fea3c81_images.jpg',
 'http://assets.myntassets.com/v1/images/style/properties/Puma-Women-Black-Shorts_e0aa48a440adafbe0a307614bfb8e551_images.jpg',
 'http://assets.myntassets.com/v1/images/style/properties/f5c08f2a59199c361b80b06ad9be1d88_images.jpg']

In [407]:
def url_to_image(url, readFlag=cv2.IMREAD_COLOR):
    # download the image, convert it to a NumPy array, and then read
    # it into OpenCV format
    resp = urlopen(url) 
    image = np.asarray(bytearray(resp.read()), dtype="uint8")
    image = cv2.imdecode(image, readFlag)

    # return the image
    return image

In [408]:
for i in final_img:
    temp_img = imutils.url_to_image(i)
    cv2.imshow('output',cv2.resize(temp_img,(512,512)))
    cv2.waitKey(0)

In [351]:
os.remove(img_link) # delete img coz that downloded file 